<a href="https://colab.research.google.com/github/ficle-fr/pix2pix_change_style/blob/colabs/colabs/train_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/common.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/descriminator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/generator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/img_generator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/train.py

#Currently from the colab branch
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/colabs/colabs/read_write_db.py


--2024-04-09 12:05:52--  https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/common.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2613 (2.6K) [text/plain]
Saving to: ‘common.py’

common.py           100%[===================>]   2.55K  --.-KB/s    in 0s      

2024-04-09 12:05:52 (37.2 MB/s) - ‘common.py’ saved [2613/2613]

--2024-04-09 12:05:52--  https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/descriminator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1480 (1.4K) [text/plain]
Savin

In [2]:
!sudo apt install libcairo2-dev pkg-config python3-dev
!pip install pycairo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  libblkid-dev libblkid1 libcairo-script-interpreter2 libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libmount1 libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev
  libxcb-render0-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils libice-doc cryptsetup-bin libsm-doc
The following NEW packages will be installed:
  libblkid-dev libcairo-script-interpreter2 libcairo2-dev libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev libxcb-render0-dev
  libxcb-shm0-dev
The

In [3]:
import tensorflow as tf
import time

from generator import Generator, generator_loss
from descriminator import Discriminator, discriminator_loss

from img_generator import img_pair_gen1
from read_write_db import write, read

In [4]:
write("temp_records100.tfrecords", 100)

In [5]:
dataset = read("./temp_records100.tfrecords")
#def map_cast(image, target):
#    image = tf.cast(image, tf.float32)
#    target = tf.cast(target, tf.float32)
#    return image, target

#dataset = dataset.map(map_cast)
#BATCH_SIZE = 5
#dataset = dataset.batch(BATCH_SIZE)



In [6]:
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print("Total number of TPU cores:", tpu.get_tpu_system_metadata().num_cores)
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.15.0


BaseException: ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!

In [38]:
print(tpu_strategy.num_replicas_in_sync)
per_replica_batch_size = BATCH_SIZE // tpu_strategy.num_replicas_in_sync

#train_dataset = tpu_strategy.experimental_distribute_datasets_from_function(
#    lambda _: dataset)
#train_dataset = tpu_strategy.experimental_distribute_datasets_from_function(
#    lambda _: dataset)

with tpu_strategy.scope():
    generator = Generator([256, 256, 3], 3)
    discriminator = Discriminator([256, 256, 3])

    generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


8


In [39]:
@tf.function
def train_multiple_steps(steps, iterator):
    def step_fn(input_image, target):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            gen_output = generator(input_image, training = True)

            disc_real_output = discriminator([input_image, target], training = True)
            disc_generated_output = discriminator([input_image, gen_output], training=True)

            gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
            disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

            generator_gradients = gen_tape.gradient(gen_total_loss,
                                                    generator.trainable_variables)
            discriminator_gradients = disc_tape.gradient(disc_loss,
                                                        discriminator.trainable_variables)

            generator_optimizer.apply_gradients(zip(generator_gradients,
                                                    generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                                        discriminator.trainable_variables))

    #for input, output in dataset:
        #print("input:", input.shape)
        #print("output:", output.shape)
        #start = time.time()
        #tpu_strategy.run(step_fn, args=(input, output))
        #print(f'\nTraining time: {time.time()-start:.2f} sec\n')
    for _ in tf.range(steps):
      inp, tar = next(iterator)
      tpu_strategy.run(step_fn, args = (inp, tar))


In [41]:
@tf.function
def train_step(input_image, target):
    def step_fn(input_image, target):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            gen_output = generator(input_image, training = True)

            disc_real_output = discriminator([input_image, target], training = True)
            disc_generated_output = discriminator([input_image, gen_output], training=True)

            gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
            disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

            generator_gradients = gen_tape.gradient(gen_total_loss,
                                                    generator.trainable_variables)
            discriminator_gradients = disc_tape.gradient(disc_loss,
                                                        discriminator.trainable_variables)

            generator_optimizer.apply_gradients(zip(generator_gradients,
                                                    generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                                        discriminator.trainable_variables))

    tpu_strategy.run(step_fn, args = (input_image, target))



In [49]:
dataset = dataset.shuffle(10000)
dataset = dataset.repeat()
BATCH_SIZE = 5
dataset = dataset.batch(BATCH_SIZE)

iter_dataset = iter(dataset)
#print(next(iter_dataset))
#train_multiple_steps(10, iter_dataset)
input_image, target = next(iter_dataset)
train_step(input_image, target)

Exception ignored in: <function Executor.__del__ at 0x79bba01263b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.framework.errors_impl.UnimplementedError: File system scheme '[local]' not implemented (file: './temp_records100.tfrecords')
Exception ignored in: <function Executor.__del__ at 0x79bba01263b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.framework.errors_impl.UnimplementedError: File system s

UnimplementedError: File system scheme '[local]' not implemented (file: './temp_records100.tfrecords')